In [12]:
from napari_workflows import Workflow

In [13]:
w = Workflow()

In [14]:
from splineslicer.skeleton.binarize import binarize_image

# binarize the image
w.set(
    "binarized",
    binarize_image,
    "input",
    channel=1,
    threshold=0.5,
    closing_size=3
)


In [15]:
from splineslicer.skeleton.skeletonize import make_skeleton

# create skeleton
w.set(
    "skeleton", make_skeleton, "binarized", min_branch_length=10
)

In [16]:
from splineslicer.spline.spline_utils import fit_spline_to_skeleton

def fit_spline(
        skeleton_output,
        segments_to_flip,
        segments_to_keep,
        order_image_shape,
        order: int = 3,
        n_ctrl_pts: int = 25,
):
    skeleton_obj = skeleton_output[1]

    spline, order_image = fit_spline_to_skeleton(
        skeleton_obj=skeleton_obj,
        segments_to_flip=segments_to_flip,
        segments_to_keep=segments_to_keep,
        order_image_shape=order_image_shape,
        order=order,
        n_ctrl_pts=n_ctrl_pts,
    )

    return spline, order_image

# fit spline
w.set(
    "spline",
    fit_spline,
    "skeleton",
    segments_to_flip=[],
    segments_to_keep=[0],
    order_image_shape=(448, 505, 505),
    order=3,
    n_ctrl_pts=25
)

In [17]:
import h5py

# load image
fpath = '/Users/kyamauch/Documents/neural_tube_analysis/full_data_20210615/converted_images_20210716/nt_25s-raw_rescaled_Probabilities.h5'

with h5py.File(fpath, 'r') as f:
    im = f['exported_data'][:]

In [18]:
# Let's print out the whole workflow
print(str(w))


Workflow:
binarized <- (<function binarize_image at 0x7fb3e73cad30>, 'input', 1, 0.5, 3)
skeleton <- (<function make_skeleton at 0x7fb3e80cb1f0>, 'binarized', 10)
spline <- (<function fit_spline at 0x7fb28d443940>, 'skeleton', [], [0], (448, 505, 505), 3, 25)



In [19]:
w.set("input", im)

print(str(w))

Workflow:
binarized <- (<function binarize_image at 0x7fb3e73cad30>, 'input', 1, 0.5, 3)
skeleton <- (<function make_skeleton at 0x7fb3e80cb1f0>, 'binarized', 10)
spline <- (<function fit_spline at 0x7fb28d443940>, 'skeleton', [], [0], (448, 505, 505), 3, 25)
input <- [[[[0.99 1.   1.   ... 1.   1.   0.99]
   [1.   1.   1.   ... 1.   1.   1.  ]
   [1.   1.   1.   ... 1.   1.   1.  ]
   ...
   [1.   1.   1.   ... 1.   1.   1.  ]
   [1.   1.   1.   ... 1.   1.   1.  ]
   [0.99 1.   1.   ... 1.   1.   0.99]]

  [[1.   1.   1.   ... 1.   1.   1.  ]
   [1.   1.   1.   ... 1.   1.   1.  ]
   [1.   1.   1.   ... 1.   1.   1.  ]
   ...
   [1.   1.   1.   ... 1.   1.   1.  ]
   [1.   1.   1.   ... 1.   1.   1.  ]
   [1.   1.   1.   ... 1.   1.   1.  ]]

  [[1.   1.   1.   ... 1.   1.   1.  ]
   [1.   1.   1.   ... 1.   1.   1.  ]
   [1.   1.   1.   ... 1.   1.   1.  ]
   ...
   [1.   1.   1.   ... 1.   1.   1.  ]
   [1.   1.   1.   ... 1.   1.   1.  ]
   [1.   1.   1.   ... 1.   1.   1.  ]]

  

In [20]:
spline_data = w.get("spline")


In [23]:
print(type(spline_data[0]))
print(type(spline_data[1]))

<class 'geomdl.BSpline.Curve'>
<class 'numpy.ndarray'>
